In [1]:
# home mac : py37pytorch

# 검색어로 크롬 실행, 실행결과의 링크 추출, 링크를 타고 각 페이지 접속, 페이지 내용 전체 크롤링, text 추출하고, 단어 리스트로 만드는 코드

import re
import requests
import pandas as pd
import openpyxl
from urllib.parse import ParseResultBytes, quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words = set(stopwords.words('english')) 


options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
# 혹은 options.add_argument("--disable-gpu")

# UserAgent값을 바꿔줍시다! 서버가 인식하지 못하도록 가상으로 headless 값 추가함ㅠ
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")


In [22]:
def google_search_result(input_word):
    baseUrl = 'https://www.google.com/search?q='

    #plusUrl = input('무엇을 검색할까요? :')
    plusUrl = input_word

    # url = baseUrl + quote_plus(plusUrl)
    url = baseUrl + plusUrl
    # 한글을 사용할 경우 :  quote_plus 적용 - URL에 막 %CE%GD%EC 이런 거 생성해줌

    driver = webdriver.Chrome(executable_path= r'./data/chromedriver_mac_ver_90', chrome_options=options)
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, features="html.parser")

    v = soup.select('.yuRUbf')

    search_title_result = []
    search_linked_contents_result = []
    for i in v:
        #print(i.select_one('.LC20lb.DKV0Md').text)
        search_title_result.append(i.select_one('.LC20lb.DKV0Md').text)
        #print(i.a.attrs['href'])
        search_linked_contents_result.append(i.a.attrs['href'])
        #print()
        
    print(search_linked_contents_result)


    # search_linked_contents_result 의 각 링크로 접속하여 해당 내용을 모두 text로 크롤링한 후, 단어만 추리고, 다시 주요키워드를 추출한다.
    
    get_all_linked_web_data = []
    for linked_page in search_linked_contents_result:
        driver.get(linked_page)
        html = driver.page_source
        get_all_data = BeautifulSoup(html, features="html.parser")
        get_all_linked_web_data.append(get_all_data)
        
    body = re.search('<body.*/body>', html, re.I|re.S)
    if (body is None):
        print ("No <body> in html")
        exit()
            
    body = body.group()
    #print(body)
    
    # 추출된 정보 클린징
    korean = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')#한글제거

    item_extract = str(body).replace('\n', ' ')
    item_extract = re.sub('<span.*?>.*?</span>', ' ', item_extract)
    item_extract = re.sub('<b>.*?</b>', ' ', item_extract)    
    item_extract = re.sub('<.*?>', ' ', item_extract)        
    item_extract = item_extract.replace('\t', ' ')
    item_extract = re.sub(korean, '', item_extract)
    item_extract = re.sub('[-=.#/?:$}]', ' ', item_extract)
    item_extract = re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』;{}()'\\‘|\(\)\[\]\<\>`\'…》]", ' ', item_extract)
    #print (item_extract)
    
    driver.close()

    return item_extract



In [23]:
get_result = google_search_result('Jeffrey Preston') 

/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


['https://ko.wikipedia.org/wiki/%EC%A0%9C%ED%94%84_%EB%B2%A0%EC%9D%B4%EC%A1%B0%EC%8A%A4', 'https://en.wikipedia.org/wiki/Jeff_Bezos', 'https://twitter.com/jeffbezos', 'https://www.indiatoday.in/technology/features/story/jeff-bezos-delivered-books-in-his-car-when-he-started-amazon-now-he-is-retiring-as-ceo-1765399-2021-02-03', 'https://www.britannica.com/biography/Jeff-Bezos', 'https://www.biography.com/business-figure/jeff-bezos', 'https://www.kings.uwo.ca/academics/faculty-info/member-profile/?doaction=getProfile&id=jpresto7', 'https://www.britannica.com/biography/Jeff-Bezos', 'http://www.jeffpreston.ca/', 'https://ca.linkedin.com/in/jeffreympreston', 'https://kr.linkedin.com/in/jeffrey-preston-1880525b', 'https://www.business-standard.com/about/who-is-jeff-bezos', 'https://www.pinterest.com/neryazabar/jeffrey-preston-bezos/']


In [24]:
get_result_str = str(get_result)
get_result_str

'      article  aside  details  figcaption  figure  footer  header  hgroup  nav  section  ul     display  block     audio  canvas  video     display  inline block     audio not  controls       display  none     html     font size  100     text size adjust  none     a hover  a active     outline  0     sub  sup     position  relative    font size  75     line height  0    vertical align  baseline    sup     top   0 5em    sub     bottom   0 25em     img     max width  100     width  auto    height  auto     vertical align  middle    border  0     ms interpolation mode  bicubic      map_canvas img     max width  none     fieldset     border  0    margin  0    padding  0     button  input  select  textarea     font size  100     margin  0    vertical align  middle    button  input     overflow  visible    line height  normal    button   moz focus inner  input   moz focus inner     padding  0    border  0    button  input type  button    input type  reset    input type  submit       cursor

In [25]:
def cleaning_data(input_data):
    remove = input_data.replace("  "," ") # 변환
    remove_ = re.sub(r"\t", " ", remove) # 제거
    remove__ = re.sub(r"\n", " ", remove_) # 제거
    remove__ = remove__.replace("   ", " ")
    remove__ = remove__.replace("  ", " ")
    remove__ = remove__.replace(" ", ",")
    remove__ = remove__.replace("…/", " ")
    remove__ = remove__.replace("…", " ")
    remove__ = remove__.replace("/", " ")
    remove__ = remove__.replace(" ", ",")
    remove__ = remove__.replace(")", ",")
    remove__ = remove__.replace("(", ",")
    preprossed = remove__.split(",") # 단어를 리스트로 변환
    #print(preprossed)
    
    # 표제어 추출, 동사는 현재형으로 변환
    lemma_list =[]
    for i in preprossed:
        lema_re = lemmatizer.lemmatize(i, pos='v') #표제어 추출, 동사는 현재형으로 변환
        lemma_list.append(lema_re)
    
    # 표제어 추출
    ext_lema = [lemmatizer.lemmatize(w) for w in preprossed]
    # 중복값을 제거하고
    rm_dupli = set(ext_lema)
    # 다시 리스트로 만들고
    re_li = list(rm_dupli)
    # 빈 값은 제거하고
    get_wd =list(filter(None, re_li))
    # 소문자로 모두 변환
    lower_wd = [i.lower() for i in get_wd]
    
    result = []
    for w in lower_wd: 
        if w not in stop_words: 
            result.append(w)
    return result

In [26]:
result_cln = cleaning_data(get_result_str)
result_cln

['web_unauth_breadcrumbs_refresh',
 'וחצי',
 'get',
 'may',
 '98',
 'search_personalization_card',
 'pixie_hf_p2p_repin_ranking_top_level',
 'sterling_cbo_delivery_estimation_error_message',
 '736px',
 'fdea5ab65166d956c460685ac4ee83c6',
 'reporter',
 'f6',
 'light\\u0026hd',
 'historia',
 'בטוויטר',
 'fatimabutool84',
 'e4d54391c4cf5b8d73fd',
 'web_eg_windows_pwa',
 'advanced',
 '6d77a2',
 '303',
 '54833ed930c7228eaed1',
 'control_05',
 'ｍｓ',
 'stefanpopowycz',
 'smartfeed_pfy_selection_experiment',
 'story_pin_recommendable_v4_social_signal_exp',
 'אינפוגרפיה',
 'pins',
 'webapp',
 'take',
 'space',
 'incident',
 'hldauphiny',
 'enabled_pickup_tw_12',
 'successful',
 'accounting',
 '140395381853008',
 '66d1e67cd19aeb52bc08',
 'however',
 'clientsortdisabled',
 '790px',
 'hf_rtc_remove_fresh_p2b_hf_local_v2',
 'pinnability_retrained_prod_2020_03_22',
 'inset',
 'provider_name',
 '28hb',
 '4249px',
 'xiaye_test_nag_501060_ab_test',
 'submitted',
 'm5xm',
 'fafafa',
 'וג',
 'mweb_target

In [27]:
# 새로운 리스트

list_par = []

# 텍스트를 가지고 있는 리스트

for i in result_cln:

    # 영어,숫자 및 공백 제거.

    text = re.sub('[^a-zA-Z0-9]',' ',i).strip()

    # 빈 리스트는 제거.

    if(text != ''):

        list_par.append(text)

In [28]:
list_par

['web unauth breadcrumbs refresh',
 'get',
 'may',
 '98',
 'search personalization card',
 'pixie hf p2p repin ranking top level',
 'sterling cbo delivery estimation error message',
 '736px',
 'fdea5ab65166d956c460685ac4ee83c6',
 'reporter',
 'f6',
 'light u0026hd',
 'historia',
 'fatimabutool84',
 'e4d54391c4cf5b8d73fd',
 'web eg windows pwa',
 'advanced',
 '6d77a2',
 '303',
 '54833ed930c7228eaed1',
 'control 05',
 'stefanpopowycz',
 'smartfeed pfy selection experiment',
 'story pin recommendable v4 social signal exp',
 'pins',
 'webapp',
 'take',
 'space',
 'incident',
 'hldauphiny',
 'enabled pickup tw 12',
 'successful',
 'accounting',
 '140395381853008',
 '66d1e67cd19aeb52bc08',
 'however',
 'clientsortdisabled',
 '790px',
 'hf rtc remove fresh p2b hf local v2',
 'pinnability retrained prod 2020 03 22',
 'inset',
 'provider name',
 '28hb',
 '4249px',
 'xiaye test nag 501060 ab test',
 'submitted',
 'm5xm',
 'fafafa',
 'mweb targeted lite app upsell',
 'real',
 'absolute',
 '4',
 '